# JAMS Creation #

This notebook creates the JAMS data given the mp3 files (containing metadata in their ID3 tags) and the actual raw annotations.

In [1]:
import eyed3
import glob
import jams
import os
from tqdm import tqdm_notebook as tqdm
import pickle

DATASET_PATH = "/Users/onieto/dev/harmonix-dataset-ismir-2019/dataset/"
BEAT_DOWNBEAT_RAW_ANN_PATH = DATASET_PATH + "beats/"
SEGMENT_RAW_ANN_PATH = DATASET_PATH + "segments/"
MP3_FILES = glob.glob("/Users/onieto/Desktop/Harmonix/final_audio/*.mp3")
JAMS_OUTPUT = "./jams/"
ROUND_PRECISION = 3

In [15]:
def add_metadata(jam, mp3_info):
    """Add metadata to the JAMS object."""
    jam.file_metadata.duration = round(mp3_info.info.time_secs, ROUND_PRECISION)
    jam.file_metadata.artist = mp3_info.tag.artist
    jam.file_metadata.release = mp3_info.tag.album
    jam.file_metadata.title = mp3_info.tag.title
    
    jam.file_metadata.identifiers = {
        "Acoustid Id": mp3_info.tag.user_text_frames.get(u"Acoustid Id").text,
        "MusicBrainz Id": mp3_info.tag.unique_file_ids.get("http://musicbrainz.org").uniq_id.decode('ascii')}
    try:
        # Not all of the tracks have an associated release
        jam.file_metadata.identifiers["MusicBrainz Release Track Id"] = \
            mp3_info.tag.user_text_frames.get(u"MusicBrainz Release Track Id").text
    except AttributeError:
        pass
    
def add_beats_and_downbeats(jam, beat_dnbeat_txt):
    """Add beat and downbeat annotations to the JAMS object."""
    with open(beat_dnbeat_txt, "r") as f:
        beat_dnbeat_data = f.readlines()
    
    # Create annotation
    ann = jams.Annotation(namespace='beat', time=0, 
                          duration=jam.file_metadata.duration)
    
    # Add annotations
    for beat_row in beat_dnbeat_data:
        time, beat_pos, bar_num = beat_row.split('\t')
        ann.append(time=round(float(time), ROUND_PRECISION), 
                   duration=0.0, 
                   confidence=1, 
                   value=int(beat_pos))
    
    # TODO: Add Annotation Metadata
    
    # Add annotation to 
    jam.annotations.append(ann)
    
    
def add_segmentation(jam, segmentation_txt):
    """Add segmentation annotations to the JAMS object."""
    with open(segmentation_txt, "r") as f:
        segment_data = f.readlines()
    
    # Create annotation
    ann = jams.Annotation(namespace='segment_open', time=0, 
                          duration=jam.file_metadata.duration)
    
    # Add annotations
    for start, end in zip(segment_data[:-1], segment_data[1:]):
        start_time, start_label = start.replace('\n', '').split(' ')
        end_time, _ = end.replace('\n', '').split(' ')
        ann.append(time=round(float(start_time), ROUND_PRECISION), 
                   duration=round(float(end_time) - float(start_time), ROUND_PRECISION), 
                   confidence=1, 
                   value=start_label)
    
    # TODO: Add Annotation Metadata
    
    # Add annotation to 
    jam.annotations.append(ann)

def create_jams(mp3_file):
    """Creates the JAMS object"""
    beat_dnbeat_txt = "{}{}".format(
        BEAT_DOWNBEAT_RAW_ANN_PATH, 
        os.path.basename(mp3_file).replace(".mp3", ".txt"))
    segment_txt = "{}{}".format(
        SEGMENT_RAW_ANN_PATH, 
        os.path.basename(mp3_file).replace(".mp3", ".txt"))
    
    # Read ID3 tags
    mp3_info = eyed3.load(mp3_file)
    
    # Create empty JAMS object and add file metadata
    jam = jams.JAMS()
    add_metadata(jam, mp3_info)
    
    # Add annotations
    add_beats_and_downbeats(jam, beat_dnbeat_txt)
#     add_est_beats(jam, mp3_file)
    add_segmentation(jam, segment_txt)
    
    # Add the beats info
    return jam

In [21]:
for mp3_file in tqdm(MP3_FILES):
    out_file = "{}{}".format(JAMS_OUTPUT, os.path.basename(mp3_file).replace(".mp3", ".jams"))
    jam = create_jams(mp3_file)
    jam.save(out_file)

GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
Invalid date: 0401
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime 

GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
Invalid date: 0302
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
Invalid date: 0611
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
Invalid date: 0305
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime ty

Invalid date: 0908
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
Invalid date: 0611
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
Invalid date: 0708
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
Invalid date: 0712
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
Invalid date: 0911
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
Invalid date: 0109
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
Invalid date: 0305
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB fr

Invalid date: 0811
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
Invalid date: 0702
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
Invalid date: 0105
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
Invalid date: 0702
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
Invalid date: 0105
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
Invalid date: 0908
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
Invalid date: 0611
GEOB fr

GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
Invalid date: 0311
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
Invalid date: 0306
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does 

GEOB frame does not contain a valid mime type
Frame 'IPLS' is not yet supported, using raw Frame to parse
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
Invalid date: 0202
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
Invalid date: 0702
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
Invalid date: 0601
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
Frame 'IPLS' is not yet supported, using raw Frame to parse
Invalid date: 0510
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a 

Invalid date: 0601
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
Frame 'IPLS' is not yet supported, using raw Frame to parse
Invalid date: 0510
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GE

GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
Invalid date: 0110
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
Invalid date: 0110
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
Invalid date: 0306
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime ty

GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
Invalid date: 0702
GEOB frame does not contain a mime 

In [4]:
with open("/Users/onieto/Downloads/2019_04_09_19_15_00_Beat_Tracking_Estimates.pkl", "rb") as f:
    all_beats = pickle.load(f)
all_beats_dict = {}
for b in all_beats:
    all_beats_dict[b[0]] = b[1]

In [19]:
def add_est_beats(jam, track_name, algo="madmom_2"):
    """Add beat and downbeat annotations to the JAMS object."""
    est_beats = all_beats_dict[os.path.basename(track_name).replace(".mp3", "")][algo]
    
    # Create annotation
    ann = jams.Annotation(namespace='beat', time=0, 
                          duration=jam.file_metadata.duration)
    
    # Add annotations
    for beat in est_beats:
        ann.append(time=round(beat, ROUND_PRECISION), 
                   duration=0.0, 
                   confidence=1, 
                   value=0)
    
    # TODO: Add Annotation Metadata
    
    # Add annotation to 
    jam.annotations.append(ann)